In [1]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from functools import reduce
import operator
import json
from pampy import match
import re
from requests_html import HTMLSession
session = HTMLSession()

In [99]:
from IPython.core.display import display, HTML
display(HTML('''
<!DOCTYPE html>
<html>
<head>
  <link href="https://fonts.googleapis.com/css?family=Roboto:100,300,400,500,700,900|Material+Icons" rel="stylesheet">
  <link href="https://cdn.jsdelivr.net/npm/vuetify/dist/vuetify.min.css" rel="stylesheet">
  <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, minimal-ui">
</head>
<body>
  <div id="app">
    <v-app>
      <v-content>
          <v-layout row>
            <v-flex xs12 sm2 offset-sm3>
              <v-card>
                <v-toolbar color="pink" dark>
                  <v-toolbar-title>Inbox</v-toolbar-title>
                </v-toolbar>

                <v-list>
                  <template v-for="(item, index) in items">
                    <v-list-tile
                      :key="item.title"
                      @click="toggle(index)"
                    >
                      <v-list-tile-content>
                        <v-list-tile-title>{{ item.title }}</v-list-tile-title>
                      </v-list-tile-content>
                    </v-list-tile>
                  </template>
                </v-list>
              </v-card>
            </v-flex>
          </v-layout>
      </v-content>
    </v-app>
  </div>

  <script src="https://cdn.jsdelivr.net/npm/vue/dist/vue.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/vuetify/dist/vuetify.js"></script>
  <script>
    new Vue({ 
        el: '#app',
        data: {
            selected: [2],
            items: [
              {
                action: '15 min',
                headline: 'Brunch this weekend?',
                title: 'Ali Connors',
                subtitle: "I'll be in your neighborhood doing errands this weekend. Do you want to hang out?"
              },
              {
                action: '2 hr',
                headline: 'Summer BBQ',
                title: 'me, Scrott, Jennifer',
                subtitle: "Wish I could come, but I'm out of town this weekend."
              },
              {
                action: '6 hr',
                headline: 'Oui oui',
                title: 'Sandra Adams',
                subtitle: 'Do you have Paris recommendations? Have you ever been?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '18hr',
                headline: 'Recipe to try',
                title: 'Britta Holt',
                subtitle: 'We should eat this: Grate, Squash, Corn, and tomatillo Tacos.'
              }
            ]        
        },
        methods: {
         toggle (index) {
            const i = this.selected.indexOf(index)

            if (i > -1) {
              this.selected.splice(i, 1)
            } else {
              this.selected.push(index)
            }
          }        
        }
    })
  </script>
</body>
</html>
'''))

In [11]:
def get_media_list():
    r = session.get('https://m.news.naver.com/newspaper/home.nhn')
    return [{'id': li.attrs['id'], 'title': li.find('img', first=True).attrs['alt']} for li in r.html.find('ul.offc_lst._headline_list > li')]

def get_article_list(media, ymd):
    url = 'https://media.naver.com/press/{}/newspaper?date={}'
    r = session.get(url.format(media, ymd))
    article_list = [{'title': article.find('strong', first=True).text, 'link': article.find('a', first=True).attrs['href']} for article in r.html.find('ul.newspaper_article_lst > li')]
    return article_list

def get_summary(article_link):
    m = match(article_link, re.compile('https://.+/.+/.+/([0-9]+)/([0-9]+)'), lambda m, a: (m, a))
    r = session.get('https://tts.news.naver.com/article/{}/{}/summary'.format(*m))
    rj = r.json()
    if 'summary' in rj:
        return rj['summary']
    return None

In [6]:
media_list = get_media_list()
media_list

[{'id': '023', 'title': '조선일보'},
 {'id': '022', 'title': '세계일보'},
 {'id': '353', 'title': '중앙SUNDAY'},
 {'id': '014', 'title': '파이낸셜뉴스'},
 {'id': '028', 'title': '한겨레'},
 {'id': '011', 'title': '서울경제'},
 {'id': '469', 'title': '한국일보'},
 {'id': '032', 'title': '경향신문'},
 {'id': '020', 'title': '동아일보'},
 {'id': '015', 'title': '한국경제'},
 {'id': '005', 'title': '국민일보'},
 {'id': '009', 'title': '매일경제'},
 {'id': '018', 'title': '이데일리'},
 {'id': '016', 'title': '헤럴드경제'},
 {'id': '081', 'title': '서울신문'},
 {'id': '029', 'title': '디지털타임스'},
 {'id': '277', 'title': '아시아경제'},
 {'id': '044', 'title': '코리아헤럴드'},
 {'id': '030', 'title': '전자신문'},
 {'id': '008', 'title': '머니투데이'},
 {'id': '021', 'title': '문화일보'}]

In [15]:
article_list = get_article_list('028', '20190504')
len(article_list)

35

In [14]:
get_summary(article_list[0]['link'])

'박상기 법무부 장관이 3일 검찰의 수사 관행을 비판하며 \'권한 재조정\'이 필요하다고 말했다.<br/><br/>이날 오전 경기도 수원에서 열린 \'수원고검 개청식 및 수원검찰청사 준공식\'에 참석한 박 장관은 기념사를 통해 "시대 상황은 변했고 국민의 시각과 의식도 달라졌다.검찰의 수사 관행은 물론이고, 권한도 견제와 균형의 원리에 맞도록 재조정돼야 한다"고 말했다.<br/><br/>지난 1일 문무일 검찰총장이 검경 수사권 조정 법안에 대해 "견제와 균형이라는 민주주의 원리에 반한다"고 비판했는데, \'견제\'와 \'균형\'이라는 단어를 그대로 써서 반박한 것이다.'

In [17]:
datetime.now().strftime('%Y%m%d')

'20190504'